# Entries by Day using Linear Regression/SVM #

## Setup

In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib, matplotlib.pyplot as plt

from sklearn import cross_validation
from sklearn import grid_search

from sklearn import linear_model
from sklearn import svm

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Setup.
% matplotlib inline

In [2]:
data = pd.read_csv("../../../../data/mbta_daily.csv", low_memory=False)
data.head()

,locationid,service_day,entries,name,line_1,line_2,lat,lon,service_datetime,fog,...,entries_weeks_ago_1,entries_weeks_ago_2,entries_weeks_ago_3,rain_predict,rain_fall_predict,snow_predict,snow_fall_predict,snow_accum,snow_accum_predict,dist_to_center
0,1002,2013-01-01 00:00:00,1892,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-01 03:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
1,1002,2013-01-02 00:00:00,5134,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-02 04:45:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
2,1002,2013-01-03 00:00:00,5733,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-03 05:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
3,1002,2013-01-04 00:00:00,6125,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-04 05:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
4,1002,2013-01-05 00:00:00,3410,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-05 04:15:00,0,...,NaN,NaN,NaN,0,0,1,0,0,0,3.404767


In [3]:
print("Original: " + str(len(data)))

data = data[(pd.DatetimeIndex(data['service_day']).weekday != 5) & (pd.DatetimeIndex(data['service_day']).weekday != 6)]

print("Remaining: " + str(len(data)))
data.head()

Original: 47901
Remaining: 34294


,locationid,service_day,entries,name,line_1,line_2,lat,lon,service_datetime,fog,...,entries_weeks_ago_1,entries_weeks_ago_2,entries_weeks_ago_3,rain_predict,rain_fall_predict,snow_predict,snow_fall_predict,snow_accum,snow_accum_predict,dist_to_center
0,1002,2013-01-01 00:00:00,1892,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-01 03:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
1,1002,2013-01-02 00:00:00,5134,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-02 04:45:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
2,1002,2013-01-03 00:00:00,5733,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-03 05:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
3,1002,2013-01-04 00:00:00,6125,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-04 05:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767
6,1002,2013-01-07 00:00:00,5998,Andrew Square,Red,NaN,42.32955,-71.05696,2013-01-07 04:00:00,0,...,NaN,NaN,NaN,0,0,0,0,0,0,3.404767


In [4]:
"""
"""
def predict(station, x_cols, predictor, parameters, ape_threshold, state, drop_outliers = False):
    # Copy the station so we don't manipulate the original
    station = station.copy()
    
    # Get the columns of the dataframe.
    cols = list(station.columns)
    
    # Determine the indices of the columns.
    y_col_indices = [0] + list(np.where([col == 'entries' for col in cols])[0] + 1)
    x_col_indices = [0] + list(np.where([col in x_cols for col in cols])[0] + 1)
    
    # Make sure none of the predictor fields are null.
    for col in x_cols:
        station = station[pd.notnull(station[col])]
    
    # Remove any entries where no one was there...
    station = station[station['entries'] > 50]
    
    # Reset the station indices, we have to reset twice so the matrix values gets the index column.
    station = station.reset_index()
    station.drop('index', axis=1, inplace=True)
    station = station.reset_index()
    
    # Get the dataframe as a matrix where the first column is the index.
    matrix = station.values
    
    # Slice so the y only contains 2 column (index, entries)
    #  and the x is a matrix that contains the index and all the predictors.
    y = matrix[:,y_col_indices]
    x = matrix[:,x_col_indices]
    
    # Split the data set into a train and test.
    x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size=0.2, random_state=state)
    
    # Convert the train and test sets into a format sklean fit() expects.
    x_train_fit = np.matrix(x_train[:,1:], dtype=np.float32)
    y_train_fit = np.array([v[0] for v in y_train[:,1:]], dtype=np.uint16)
    
    x_test_fit = np.matrix(x_test[:,1:], dtype=np.float32)
    y_test_fit = np.array([v[0] for v in y_test[:,1:]], dtype=np.uint16)
    
    # Train using a grid search based on the parameters provided.
    clf = grid_search.GridSearchCV(predictor, parameters, scoring='mean_squared_error', cv=10)
    clf.fit(x_train_fit, y_train_fit)
    
    # Determine what the best model was.
    model = clf.best_estimator_
    
    # Predict using the test set.
    y_pred_fit = model.predict(x_test_fit)
    
    # Determine the absolute percent errors.
    apes = np.abs(y_pred_fit - y_test_fit) / y_test_fit
    
    # Determine any "extreme" outliers by determining large percent errors.
    ape_indices = apes >= ape_threshold
    outlier_indices = y_test[ape_indices][:,0]
    outliers = station.iloc[outlier_indices]
    outliers['entries_predicted'] = y_pred_fit[ape_indices]
    outliers['entries_ape'] = apes[ape_indices]
    
    # Remove any percent difference that would cause an error in calculating the mean.
    apes = apes[apes < float('+inf')]
    if (drop_outliers):
        apes = apes[apes < ape_threshold]
    
    return model, apes, outliers

In [5]:
def find_outliers(station, x_cols, model):
    outliers = pd.DataFrame()

    for index, row in station.iterrows():
        y = row['entries']
        x = row.as_matrix(x_cols)
        
        nan = False
        for i in x:
            if np.isnan(i):
                nan= True
                break
        if nan:
            continue
        
        y_pred = model.predict(x)

        ape = np.abs(y_pred[0] - y) / y

        if (ape > .2):
            row['entries_predict'] = y_pred[0]
            row['entries_ape'] = ape

            outliers = outliers.append(row)
    
    return outliers

In [41]:
def features_test(stationids, x_cols):
    accuracies = []
    outliers_count = []
    
    for stationid in stationids:
        station = data[data['locationid'] == stationid]

        #predictor = svm.SVR()
        #parameters = {'kernel': ('linear', 'rbf'), 'C': range(1, 2)}
        predictor = linear_model.LinearRegression()
        parameters = {}
        
        # Train and predict X times, each with a different train/test set using a random state.
        mean_apes = []
        mean_outliers = []
        for state in xrange(10):
            model, apes, outliers = predict(station, x_cols, predictor, parameters, .2, state, True)
            #outliers.to_csv("outliers-tmp-" + str(stationid) + ".csv", index=False)
            #print(np.mean(apes))
            mean_apes.append(np.mean(apes))
            mean_outliers.append(len(outliers))

        #outliers = find_outliers(station, x_cols, model)
        #outliers.to_csv("outliers-" + str(stationid) + ".csv", index=False)
        
        accuracies.append(1 - np.mean(mean_apes))
        outliers_count.append(np.mean(mean_outliers))
        
        print(station.iloc[0]["name"] + ": " + str(accuracies[-1]))# + " (" + str(outliers_count[-1]) + ")"
    
    return accuracies, outliers_count

In [42]:
stationids = [
    1020, # Braintree
    1032, # Alewife
    1035, # Harvard
    1039, # Downtown
    1043, # Ashmont
    1059, # Kenmore
    1075, # North
]

In [43]:
accuracies, outliers_count = features_test(stationids, ['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'entries_weeks_ago_1'])
print("")
print("Mean Acc.: " + str(np.mean(accuracies)))
print("Var Ac.: " + str(np.var(accuracies)))
print("")
#print("Mean Outliers.: " + str(np.mean(outliers_count)))

Braintree: 0.940939670925
Alewife: 0.945968636398
Harvard: 0.945009409866
Downtown Crossing: 0.945641917908
Ashmont: 0.934765053715
Kenmore Square: 0.923819531105
North Station: 0.920146811654

Mean Acc.: 0.93661300451
Var Ac.: 9.94988830585e-05



In [37]:
accuracies, outliers_count = features_test(stationids, ['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'entries_weeks_ago_1', 'snow_fall'])
print("")
print("Mean Acc.: " + str(np.mean(accuracies)))
print("Var Ac.: " + str(np.var(accuracies)))
print("")
#print("Mean Outliers.: " + str(np.mean(outliers_count)))

Braintree: 0.824133935605 (10.9)
Alewife: 0.802462865258 (10.3)
Harvard: 0.851474464359 (10.5)
Downtown Crossing: 0.619149123287 (9.1)
Ashmont: 0.846935852425 (8.5)
North Station: 0.716187723508 (16.1)

Mean Acc.: 0.776723994074
Var Ac.: 0.00698700164406



In [38]:
accuracies, outliers_count = features_test(stationids, ['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'entries_weeks_ago_1', 'snow'])
print("")
print("Mean Acc.: " + str(np.mean(accuracies)))
print("Var Ac.: " + str(np.var(accuracies)))
print("")
#print("Mean Outliers.: " + str(np.mean(outliers_count)))

Braintree: 0.813675263564 (11.2)
Alewife: 0.792026420638 (11.3)
Harvard: 0.841790863532 (11.1)
Downtown Crossing: 0.447795204618 (9.1)
Ashmont: 0.829769979854 (10.0)
North Station: 0.711130771263 (16.6)

Mean Acc.: 0.739364750578
Var Ac.: 0.0187949126081



In [39]:
accuracies, outliers_count = features_test(stationids, ['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'entries_weeks_ago_1', 'snow_accum'])
print("")
print("Mean Acc.: " + str(np.mean(accuracies)))
print("Var Ac.: " + str(np.var(accuracies)))
print("")
#print("Mean Outliers.: " + str(np.mean(outliers_count)))

Braintree: 0.82811880225 (10.8)
Alewife: 0.802076521309 (10.1)
Harvard: 0.849448148284 (10.6)
Downtown Crossing: 0.59284369088 (9.7)
Ashmont: 0.843927997629 (9.1)
North Station: 0.712857503969 (16.6)

Mean Acc.: 0.771545444053
Var Ac.: 0.00846989130386



In [40]:
accuracies, outliers_count = features_test(stationids, ['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'entries_weeks_ago_1', 'snow_fall', 'snow_accum'])
print("")
print("Mean Acc.: " + str(np.mean(accuracies)))
print("Var Ac.: " + str(np.var(accuracies)))
print("")
#print("Mean Outliers.: " + str(np.mean(outliers_count)))

Braintree: 0.82795100997 (10.6)
Alewife: 0.802781192496 (10.0)
Harvard: 0.851274557388 (10.5)
Downtown Crossing: 0.629702943666 (9.2)
Ashmont: 0.845384386198 (8.8)
North Station: 0.715182116487 (16.5)

Mean Acc.: 0.778712701034
Var Ac.: 0.00649234715203

